# core

> Main codebase for fetching and saving RSS feeds

In [1]:
# | default_exp core


In [2]:
# | hide
from nbdev.showdoc import *


In [14]:
# | export

import reader
from typing import *
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s)
    return s.text


In [60]:
# | export

class Feed:
    """RSS feed class 
    """

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)


In [212]:
# | export

class PyNewsReader:
    def __init__(self, dbpath=None, feeds=List[Feed]):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self._reader = reader.make_reader(
            "db.sqlite", plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        self._reader.enable_search()
        self._feed_names = {}

    def _print_entries(self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = Text(
                        "Date: " + e.published.isoformat()[:10], justify="center"
                    )
                else:
                    published_date = Text("Date: Unknown", justify="center")
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = self._get_feed_title(e.original_feed_url)
                print(
                    Panel(
                        published_date
                        + "\n\n"
                        + Text(strip_html(e.summary) + "\n", justify=None),
                        title=f"[link={e.link}]{e.title}[/link]",
                        subtitle=feed_title,
                    )
                )
                print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed

        Args:
            url (str): URL of pynewsreader feed

        Returns:
            str: Display title
        """
        if url in self._feed_names:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()

    def get_entries(self, important: bool = None, read: Union[None, bool] = None, limit: int = 10):
        """Get entries in reader.Entry format

        Args:
            read (reader.Entry.read, optional): Filter on `read` status (None, True, False). Defaults to None.
            limit (int, optional): Number of entries to return. Defaults to 10.

        Returns:
            List[reader.Entry]: List of entries
        """
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def add_feed(self, feed: Feed):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        self._reader.add_feed(feed.url, exist_ok=True)
        if feed.name:
            self._feed_names[feed.url] = feed.name

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def list_feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def show(self, limit: int = 5, read: bool = None, important: bool = None, mark_as_read: bool = False):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(self.get_entries(
            read=read, important=important, limit=limit*2), limit=limit, mark_as_read=mark_as_read)

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries([self._search_to_entry(i) for i in self._reader.search_entries(
            query)], mark_as_read=mark_as_read, limit=limit)

    def mark_important(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_important(entry)

    def tag_entry(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Tag an entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
            tag_value (Dict, optional): Value of tag. Defaults to None.
        """
        reader.Reader.set_tag(entry, tag_key, tag_value)


In [213]:
r = PyNewsReader()


## Add Feeds

In [214]:
# With names
for i in [
    Feed(url='https://ricochet.media/en/feed', name="Richochet Media"),
    Feed(url='https://thetyee.ca/rss2.xml', name="The Tyee"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
         name="Toronto Star - Top Stories"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
         name="Toronto Star - Investigations"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
         name="Toronto Star - Editorials"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://www.macleans.ca/feed/",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))


## List Feeds

In [215]:
r.list_feeds()


['Richochet Media',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans.ca',
 'The Narwhal',
 'The Tyee',
 'Toronto Star - Investigations',
 'Toronto Star - Editorials',
 'Toronto Star - Top Stories']

## Update Feeds

In [167]:
r.update()


## Get Entries

In [216]:
for i in r.get_entries(limit=1):
    print(i)


Entry(
    id='1.6686447',
    updated=None,
    title='MPs pass bill mandating internet giants pay for news. Meta threatens to block content',
    link='https://www.cbc.ca/news/politics/bill-c18-internet-news-meta-1.6686447?cmp=rss',
    author='The Canadian Press',
    published=datetime.datetime(2022, 12, 15, 0, 40, 2, tzinfo=datetime.timezone.utc),
    summary='<img alt="Senate Committee Bill C-11 20221122" height="259" 
src="https://i.cbc.ca/1.6686463.1671064617!/fileImage/httpImage/image.JPG_gen/derivatives/16x9_460/senate-committee
-bill-c-11-20221122.JPG" title="Minister of Canadian Heritage Pablo Rodriguez prepares to appear before the Senate 
Committee on Transport and Communications on Bill C-11, in the Senate of Canada Building in Ottawa, on Tuesday, 
Nov. 22, 2022. THE CANADIAN PRESS/Justin Tang" width="460" />                <p>The House of Commons passed a 
Liberal bill on Wednesday designed to require web giants to compensate journalism outfits for reposting their 
content, and Meta is once again threatening to remove news content from Facebook in Canada.</p>',
    content=(),
    enclosures=(),
    read=True,
    read_modified=datetime.datetime(2022, 12, 15, 1, 13, 16, 72777, tzinfo=datetime.timezone.utc),
    important=False,
    important_modified=None,
    added=datetime.datetime(2022, 12, 15, 1, 13, 5, 405940, tzinfo=datetime.timezone.utc),
    added_by='feed',
    last_updated=datetime.datetime(2022, 12, 15, 1, 13, 5, 405940, tzinfo=datetime.timezone.utc),
    original_feed_url='https://rss.cbc.ca/lineup/topstories.xml',
    feed=Feed(
        url='https://rss.cbc.ca/lineup/topstories.xml',
        updated=datetime.datetime(2022, 12, 15, 0, 40, 2, tzinfo=datetime.timezone.utc),
        title='CBC | Top Stories News',
        link='http://www.cbc.ca/news/?cmp=rss',
        author=None,
        subtitle='FOR PERSONAL USE ONLY',
        version='rss20',
        user_title=None,
        added=datetime.datetime(2022, 12, 7, 0, 40, 58, 106810, tzinfo=datetime.timezone.utc),
        last_updated=datetime.datetime(2022, 12, 15, 1, 13, 5, 405940, tzinfo=datetime.timezone.utc),
        last_exception=None,
        updates_enabled=True
    )
)

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [217]:
r.show(read=True, limit=10, mark_as_read=False)


╭───────────── ]8;id=780204;https://www.cbc.ca/news/politics/bill-c18-internet-news-meta-1.6686447?cmp=rss\MPs pass bill mandating internet giants pay for news. Meta threatens to block content]8;;\ ─────────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│       The House of Commons passed a Liberal bill on Wednesday designed to require web giants to compensate      │
│ journalism outfits for reposting their content, and Meta is once again threatening to remove news content from  │
│                                               Facebook in Canada.                                               │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭───────────────────────────────────────── ]8;id=262807;https://www.cbc.ca/news/world/monkeys-in-hot-water-in-japan-1.6686499?cmp=rss\Monkeys in hot water in Japan]8;;\ ─────────────────────────────────────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│  The Hakodate Tropical Botanical Garden in Hokkaido, Japan, has set out its annual winter bath using hot water  │
│   from a nearby spring to give monkeys from its monkey park a chance to stay warm. Despite temperatures below   │
│  zero Celsius freezing, the primates can stay relaxed and toasty while visitors take photos and toss food into  │
│                                              the penned-off area.                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

╭─ ]8;id=368548;https://www.thestar.com/sports/worldcup/opinion/2022/12/14/moroccos-magnificent-world-cup-run-comes-to-an-end-against-france-and-the-stage-is-set-for-a-piece-of-football-history.html\Bruce Arthur: Morocco’s magnificent World Cup run comes to an end against France, and the stage is set for a p]8;;\─╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│ Lionel Messi — in his last Cup — will be up in the final against Kylian Mbappé and a French team which has worn │
│                                             winning like a perfume.                                             │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star - Top Stories ───────────────────────────────────────────╯

╭───────── ]8;id=816207;https://www.cbc.ca/news/canada/ottawa/council-budget-directions-2-5-percent-1.6685803?cmp=rss\Council gives green light to draft budget with 2.5% tax hike cap — but one third not on board]8;;\ ─────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│  Mayor Mark Sutcliffe may have run on a promise not to increase property taxes more than 2.5 per cent, but one  │
│ third of the newly elected council has not signed onto that pledge, worried that it won't provide the city with │
│                                     the money it needs to serve residents.                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | Ottawa News ───────────────────────────────────────────────╯

╭─────── ]8;id=587695;https://www.cbc.ca/news/canada/saskatchewan/turpel-lafond-honorary-degrees-1.6684340?cmp=rss\Rescind Turpel-Lafond's honorary degrees or we'll return ours, say high-profile Indigenous women]8;;\ ────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│  Retired Cree lawyer Michelle Good says that if Simon Fraser University does not revoke the honorary degree it  │
│             granted Mary Ellen Turpel-Lafond, then Good will be forced to make a painful decision.              │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭─ ]8;id=605661;https://www.thestar.com/politics/federal/2022/12/12/anita-anand-misses-academic-life-but-says-shes-the-one-to-finally-tackle-sexual-misconduct-in-the-canadian-military.html\Anita Anand misses academic life — but says she’s the one to finally tackle sexual misconduct in the Canadian ]8;;\─╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│ The defence minister says she misses what she called “the solitude of academic life” — but it doesn’t mean she  │
│                                     has one foot out the door of politics.                                      │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star - Top Stories ───────────────────────────────────────────╯

╭────────────── ]8;id=92654;https://www.cbc.ca/radio/asithappens/sandy-hook-animal-sanctuary-1.6685993?cmp=rss\How the mother of a Sandy Hook victim honours her daughter's deep love for animals]8;;\ ───────────────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│     Catherine Hubbard loved animals with all her heart. So her mother runs a sanctuary in her name. Jennifer    │
│       Hubbard talks to As It Happens host Nil Köksal on the 10th anniversary of the Sandy Hook shooting.        │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

╭─────────────────────────── ]8;id=565880;https://www.thestar.com/news/gta/2022/12/14/canada-wide-warrant-issued-for-suspects-in-markham-murder.html\Canada-wide warrant issued for suspects in Markham murder]8;;\ ───────────────────────────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│     York Regional Police says a 37-year-old woman and a 38-year-old man are considered armed and dangerous.     │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star - Top Stories ───────────────────────────────────────────╯

╭────── ]8;id=993130;https://www.thestar.com/politics/federal/2022/12/14/19000-thats-how-many-models-of-guns-will-be-available-despite-new-bans-trudeau-government-says.html\19,000. That’s how many models of guns will be available despite new bans, Trudeau government says]8;;\ ───────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│   Rural Economic Development Minister Gudie Hutchings offered the number as the Liberals tried to reframe the   │
│                              debate around controversial amendments to Bill C-21.                               │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star - Top Stories ───────────────────────────────────────────╯

╭─────────────── ]8;id=376311;https://www.cbc.ca/news/canada/thunder-bay/ryan-reynolds-dennis-franklin-cromarty-high-school-interview-1.6686213?cmp=rss\Meet the First Nations students who scored an interview with actor Ryan Reynolds]8;;\ ────────────────╮
│                                                Date: 2022-12-14                                                 │
│                                                                                                                 │
│    Students with the Dennis Franklin Cromarty high school media club landed a big interview Wednesday — actor   │
│                                                 Ryan Reynolds.                                                  │
│                                                                                                                 │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

## Search Entries

In [218]:
r.search("fusion")


╭──────────────────────── ]8;id=91903;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\Here's why a breakthrough in nuclear fusion is such a big deal]8;;\ ─────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

╭─────────────────────────── ]8;id=388027;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\Fusion energy 'breakthrough' revealed by U.S. scientists]8;;\ ────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│      U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the      │
│                 decades-long quest to harness fusion, the energy that powers the sun and stars.                 │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────── ]8;id=911046;https://www.thestar.com/news/canada/2022/12/13/scientists-reveal-holy-grail-breakthrough-to-create-limitless-clean-energy.html\Scientists reveal ‘holy grail’ breakthrough to create limitless clean energy]8;;\ ──────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  Scientists have taken the first step towards cheap, clean and near unlimited power by harnessing the power of  │
│                                the sun, announced the U.S. Department of Energy.                                │
│                                                                                                                 │
╰────────────────────────────────────────── Toronto Star - Top Stories ───────────────────────────────────────────╯

In [219]:
# | hide

# To Do:
# * Command line interface
# * User interface
# * Tags


In [220]:
# | hide
import nbdev

nbdev.nbdev_export()
